In [1]:
%matplotlib inline
import os
from pathlib import Path

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import trange, tqdm

from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
import lightgbm as lgb

from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats

from fastai.datasets import Config

import warnings
warnings.filterwarnings("ignore")

base_path = Config.data_path()

# Competiton files setup

In [2]:
data_path = base_path/'LANL_Earthquake_Prediction'
competition_name = 'LANL-Earthquake-Prediction'
data_path

PosixPath('/home/krzysiek/.fastai/data/LANL_Earthquake_Prediction')

In [3]:
train_prcessed_segments_path = data_path/'train'/'processed_df'
test_prcessed_segments_path = data_path/'test_processed_df'

In [4]:
train_prcessed_segments_path.ls()

[PosixPath('/home/krzysiek/.fastai/data/LANL_Earthquake_Prediction/train/processed_df/no_overlap_time&freq_features.csv')]

In [5]:
test_prcessed_segments_path.ls()

[PosixPath('/home/krzysiek/.fastai/data/LANL_Earthquake_Prediction/test_processed_df/time&freq_features.csv')]

In [15]:
# training_set_step_10000_only_time = pd.read_csv(train_prcessed_segments_path/'step_10000_only_time_features.csv')
# training_set_no_overlap_only_time = pd.read_csv(train_prcessed_segments_path/'no_overlap_only_time_features.csv')
training_set_no_overlap_time_freq = pd.read_csv(train_prcessed_segments_path/'no_overlap_time&freq_features.csv')

In [16]:
test_set_time_freq = pd.read_csv(test_prcessed_segments_path/'time&freq_features.csv')

In [8]:
training_set_no_overlap_time_freq.head()

,target,mean,std,max,min,mean_change_abs,mean_change_rate,abs_max,std_first_50000,std_last_50000,...,Spectrogram_max_value_in_0.034304_s,Spectrogram_max_freq_in_0.034304_s,Spectrogram_max_value_in_0.035312_s,Spectrogram_max_freq_in_0.035312_s,Spectrogram_max_value_in_0.03632_s,Spectrogram_max_freq_in_0.03632_s,Spectrogram_max_value_in_0.037328_s,Spectrogram_max_freq_in_0.037328_s,PSD_peak_value,PSD_peak_freq
0,1.430797,4.884113,5.101106,104.0,-98.0,-0.000080,74836.577199,104.0,6.488552,3.664663,...,0.000105,125000.0,0.000065,218750.0,0.000032,296875.0,0.000035,218750.0,0.000260,250000.0
1,1.391499,4.725767,6.588824,181.0,-154.0,0.000000,74891.736232,181.0,7.305233,5.493071,...,0.000085,218750.0,0.000085,250000.0,0.000153,265625.0,0.000018,140625.0,0.000472,250000.0
2,1.353196,4.906393,6.967397,140.0,-106.0,-0.000013,75099.224451,140.0,6.104836,8.603696,...,0.001196,62500.0,0.000723,218750.0,0.001963,281250.0,0.000027,703125.0,0.000501,250000.0
3,1.313798,4.902240,6.922305,197.0,-199.0,0.000000,74933.991879,199.0,6.238109,5.652442,...,0.000067,140625.0,0.000019,140625.0,0.000033,265625.0,0.000086,250000.0,0.000485,250000.0
4,1.274400,4.908720,7.301110,145.0,-126.0,-0.000007,75010.016046,145.0,5.323830,7.694506,...,0.000028,250000.0,0.000025,265625.0,0.000146,171875.0,0.002869,250000.0,0.000359,93750.0


In [49]:
# training_set_no_overlap_only_time.drop(['Unnamed: 0'], axis=1, inplace=True)
test_set_time_freq['name']

0       seg_4743ab
1       seg_f86c41
2       seg_2d92f0
3       seg_1a8f2c
4       seg_8509db
5       seg_9b7ef8
6       seg_c654e7
7       seg_d59e4e
8       seg_25cca7
9       seg_22e509
10      seg_407b2b
11      seg_57908c
12      seg_268249
13      seg_15c9f9
14      seg_6f2222
15      seg_477c83
16      seg_32ad0f
17      seg_f7050a
18      seg_0968f1
19      seg_b36650
20      seg_61f504
21      seg_5254ce
22      seg_2c3203
23      seg_90c258
24      seg_5311d1
25      seg_62a403
26      seg_d398df
27      seg_1ef708
28      seg_533613
29      seg_218049
           ...    
2594    seg_41be7d
2595    seg_c42490
2596    seg_0c89ce
2597    seg_c901c0
2598    seg_b0a794
2599    seg_4c18e2
2600    seg_efc5fb
2601    seg_fde767
2602    seg_c08d36
2603    seg_a68007
2604    seg_1201e8
2605    seg_d516e3
2606    seg_f7290f
2607    seg_ab2a78
2608    seg_6d35cd
2609    seg_ff1a62
2610    seg_61219c
2611    seg_92be9f
2612    seg_5467c8
2613    seg_ef74dc
2614    seg_280863
2615    seg_

In [32]:
training_set_step_10000_only_time.drop(['Unnamed: 0'], axis=1, inplace=True)
training_set_step_10000_only_time.head()

,target,mean,std,max,min,mean_change_abs,mean_change_rate,abs_max,abs_min,std_first_50000,...,std_roll_mean_1000,max_roll_mean_1000,min_roll_mean_1000,q01_roll_mean_1000,q05_roll_mean_1000,q95_roll_mean_1000,q99_roll_mean_1000,av_change_abs_roll_mean_1000,av_change_rate_roll_mean_1000,abs_max_roll_mean_1000
0,1.430797,4.884113,5.101106,104.0,-98.0,-0.000080,74836.577199,104.0,0.0,6.488552,...,0.295715,5.629,3.896,4.072,4.379,5.338,5.484,-1.704698e-06,74222.343443,5.629
1,1.427600,4.857127,4.324007,52.0,-56.0,0.000007,74890.988734,56.0,0.0,4.506360,...,0.290901,5.629,3.896,4.072,4.375,5.320,5.483,-1.906040e-06,74333.053867,5.629
2,1.425498,4.837627,5.334216,181.0,-154.0,-0.000027,74986.731761,181.0,0.0,4.680861,...,0.298291,5.667,3.412,4.069,4.360,5.320,5.483,-6.107383e-07,74474.087926,5.667
3,1.423396,4.811160,5.322245,181.0,-154.0,0.000033,74989.309697,181.0,0.0,4.748028,...,0.303731,5.667,3.412,4.069,4.360,5.320,5.483,-3.402685e-06,74346.540137,5.667
4,1.420198,4.792853,5.418203,181.0,-154.0,0.000027,75145.482449,181.0,0.0,4.725781,...,0.302770,5.667,3.412,4.069,4.359,5.320,5.483,-3.221477e-07,74551.377120,5.667


In [86]:
feature_scores_path = data_path/'feature_evaluation'/'single_fueature_scores.csv'
feature_scores = pd.read_csv(feature_scores_path)
# best_22_features = list(feature_scores.sort_values('score')[:22]['feature_name'])

In [77]:
sorted_features = list(feature_scores.sort_values('score')['feature_name'])
print(f"# of features: {len(sorted_features)}")

# of features: 137


In [78]:
best_70_features = sorted_features[:70]
best_22_features = sorted_features[:22]

In [17]:
# Current training and test sets
df_train = training_set_no_overlap_time_freq
df_test = test_set_time_freq

df_test_segment_names = df_test['name']
df_test.drop(['name'], axis=1, inplace=True)

In [80]:
df_test = df_test[best_22_features]
best_22_features.append('target')
df_train = df_train[best_22_features]
df_train.head()

,q05_roll_std_100,q05_roll_std_1000,q05_roll_std_10,q01_roll_std_100,q01_roll_std_10,q01_roll_std_1000,mad,MA_400MA_std_mean,q95,q95,...,ave_roll_std_100,ave_roll_std_1000,MA_700MA_std_mean,Hilbert_mean,q05,q01,q99,abs_q99,q95_roll_std_1000,target
0,2.475639,2.706474,1.636392,2.302634,1.264911,2.616094,3.263401,4.155806,11.0,11.0,...,4.050450,4.288590,4.229416,7.027028,-2.0,-8.0,18.0,20.0,8.185756,1.430797
1,2.475965,2.674879,1.646545,2.300285,1.286684,2.612482,3.574302,4.608579,12.0,12.0,...,4.436359,4.843486,4.735352,7.380383,-2.0,-11.0,21.0,24.0,10.544982,1.391499
2,2.538591,2.761534,1.686548,2.374613,1.316561,2.660178,3.948411,5.122560,13.0,13.0,...,4.917334,5.423013,5.283180,8.016930,-3.0,-15.0,26.0,30.0,14.845834,1.353196
3,2.496442,2.716991,1.649916,2.330539,1.269296,2.624962,3.647117,4.698889,12.0,12.0,...,4.533343,4.939280,4.826369,7.606850,-2.0,-12.0,22.0,26.0,11.715642,1.313798
4,2.491521,2.719174,1.646545,2.314731,1.269296,2.628699,3.826052,4.910515,12.0,12.0,...,4.761149,5.121868,5.023478,7.895403,-2.0,-15.0,26.0,32.0,13.923676,1.274400


In [33]:
# Switch only train set
df_train = training_set_step_10000_only_time

In [9]:
print(f"Train length: {len(df_train)}")
print(f"Test length: {len(df_test)}")

Train length: 4178
Test length: 2624


We will use LightGBM as a model to predict target for test dataset. This is a tree-based model, which cannot see frequency of the feature (number of occurrences of particular value in a feature). It turns out that this is important in this case, so frequencies needs to be added as separate feature. 

# Model Training
## 1st Layer of ensamble

In [18]:
train_data = df_train.drop('target', axis=1)
targets = df_train['target']

## Scaling
scaler.fit(train_data)
train_data = pd.DataFrame(scaler.transform(train_data))
df_test = pd.DataFrame(scaler.transform(df_test))
##

In [24]:
param = {
    'learning_rate': 0.055,
    'num_leaves': 8,
    'metric':'mean_absolute_error',
    'boost_from_average':'false',
    'feature_fraction': 0.7, # This is important; we must use ALL features in every iteration to make sure that feature freq will be used
    'max_depth': 3,
    'objective': 'regression',
#     'verbosity': -10
    'boosting': 'gbdt',
    'lambda_l1': 0.45,
    'lambda_l2': 0.02
}

In [25]:
%%time

from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

nbr_of_folds = 10
nbr_of_rounds = 10000

folds = KFold(
    n_splits=nbr_of_folds,
    shuffle=True,
    random_state=2019
)

scaler = StandardScaler()

out_of_fold_predictions = np.zeros(len(df_train))
test_predictions = np.zeros(len(df_test))


folds_predictions = np.zeros(len(df_test))
training_results = dict()

for fold_nbr, (train_idx, valid_idx) in enumerate(folds.split(train_data, targets)):
    fold_train_data = train_data.iloc[train_idx]
    fold_train_target = targets.iloc[train_idx]
        
    fold_valid_data = train_data.iloc[valid_idx]
    fold_valid_target = targets.iloc[valid_idx]

    lgb_fold_train = lgb.Dataset(fold_train_data, label=fold_train_target)
    lgb_fold_valid = lgb.Dataset(fold_valid_data, label=fold_valid_target, reference=lgb_fold_train)

    model = lgb.train(
        param, 
        lgb_fold_train, 
        nbr_of_rounds, 
        valid_sets=[lgb_fold_train, lgb_fold_valid], 
       early_stopping_rounds=500,
        verbose_eval=False, 
        evals_result=training_results,
#        verbose_eval=5000
    )
    
#     print(training_results)
    scores = training_results['valid_1']['l1']
    best_score = min(scores)
    best_round = scores.index(min(scores))
#     print(f"Fold #{fold_nbr}: Best l1: {best_score} ({best_round} iteration)")

    out_of_fold_predictions[valid_idx] = model.predict(fold_valid_data, num_iteration=best_round)
    folds_predictions += model.predict(df_test, num_iteration=best_round)

test_predictions = folds_predictions/nbr_of_folds

print(f"Final score: {mean_absolute_error(targets.values, out_of_fold_predictions)}")

Final score: 2.0408151415932236
CPU times: user 1min, sys: 100 ms, total: 1min
Wall time: 15.2 s


In [26]:
test_predictions

array([7.795035, 4.613451, 9.083567, 8.252479, ..., 6.017693, 6.334153, 3.957822, 7.277506])

In [38]:
np.mean(targets)

5.663420702760496

Best 22 features

Without scaler: 2.07108839172104 (LB 1.614)
With scaler: 2.0674590671564608 

Best 70 features

Without scaler: 2.0766704829894915 (LB 1.604)
With scaler: 2.0746327066782286

All features

Without scaler:  2.0640357531770532 (LB 1.554?)
With scaler: 2.0634338328764064

In [103]:
np.save(data_path/'oof', out_of_fold_predictions)
np.save(data_path/'ensemble_1st_lvl', test_predictions)

## 2nd Layer of ensamble

In [109]:
from sklearn.linear_model import LogisticRegression

X_train, X_valid, Y_train, Y_valid = train_test_split(
    out_of_fold_predictions, 
    targets.values, 
    test_size=0.15, 
    random_state=2019
)

ensamble_second_layer = LogisticRegression(n_jobs=-1)
ensamble_second_layer.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn', n_jobs=-1,
          penalty='l2', random_state=None, solver='warn', tol=0.0001,
          verbose=0, warm_start=False)

In [118]:
probs = ensamble_second_layer.predict_proba(X_valid)
roc_auc_score(Y_valid, probs[:,1])

0.9040947053429751

In [115]:
import statsmodels.api as sm

logr = sm.Logit(targets, out_of_fold_predictions)
logr = logr.fit(disp=0)
ensemble_preds = logr.predict(out_of_fold_predictions)
ensemble_auc = roc_auc_score(targets, ensemble_preds)  
print('##################')
print('Combined Model with magic Val_AUC=',round(ensemble_auc,5))

##################
Combined Model with magic Val_AUC= 0.91402


In [117]:
lgb_ensemble_train = lgb.Dataset(X_train, label=Y_train)
lgb_ensemble_valid = lgb.Dataset(X_valid, label=Y_valid, reference=lgb_ensemble_train)

model = lgb.train(
    param, 
    lgb_ensemble_train, 
    50000, 
    valid_sets=[lgb_ensemble_train, lgb_ensemble_valid], 
    early_stopping_rounds=500,
#     verbose_eval=False, 
#             evals_result=training_results
    verbose_eval=1000
    )

# ensemble_predicions = model.predict(fold_valid_data)
# folds_predictions += model.predict(df_test, num_iteration=best_round)


Training until validation scores don't improve for 500 rounds.
[1000]	training's auc: 0.893676	valid_1's auc: 0.878561
[2000]	training's auc: 0.912475	valid_1's auc: 0.895864
[3000]	training's auc: 0.921207	valid_1's auc: 0.902603
[4000]	training's auc: 0.926821	valid_1's auc: 0.906273
[5000]	training's auc: 0.931289	valid_1's auc: 0.908226
[6000]	training's auc: 0.935035	valid_1's auc: 0.909387
[7000]	training's auc: 0.938219	valid_1's auc: 0.910122
[8000]	training's auc: 0.94106	valid_1's auc: 0.910847
[9000]	training's auc: 0.943547	valid_1's auc: 0.911536
[10000]	training's auc: 0.945812	valid_1's auc: 0.912033
[11000]	training's auc: 0.947898	valid_1's auc: 0.912439
[12000]	training's auc: 0.949817	valid_1's auc: 0.912743
[13000]	training's auc: 0.951595	valid_1's auc: 0.912847
Early stopping, best iteration is:
[12584]	training's auc: 0.950872	valid_1's auc: 0.91289


# Final prediction and submission to Kaggle

In [27]:
submission_df = pd.DataFrame({'seg_id': df_test_segment_names, 'time_to_failure': test_predictions})

In [28]:
submission_df.head()

,seg_id,time_to_failure
0,seg_4743ab,7.795035
1,seg_f86c41,4.613451
2,seg_2d92f0,9.083567
3,seg_1a8f2c,8.252479
4,seg_8509db,4.862505


In [29]:
submission_file_name = "time_and_freq_features_new_parameters_fixed"

submission_path = data_path/'submissions'
submission_path.mkdir(exist_ok=True)
submission_file = submission_path/f"{submission_file_name}.csv"
submission_df.to_csv(submission_file, index=False)

In [31]:
# Only 2 submission allowed per day!
!kaggle competitions submit {competition_name} -f {submission_file} -m "LGBM (optimized, fixed scaling), all time and freq features, with scaling"

100%|██████████████████████████████████████| 74.7k/74.7k [00:07<00:00, 10.5kB/s]
403 - Your team has used its submission allowance (2 of 2). This resets at midnight UTC (4.6 hours from now).
